In [1]:
import pandas as pd 
import numpy as np

In [2]:
movies = pd.read_csv('data/data_marvel.csv')

In [3]:
sales_2008 = pd.read_csv('data/sales_2008.csv')
sales_2009 = pd.read_csv('data/sales_2009.csv')
sales_2010 = pd.read_csv('data/sales_2010.csv')
sales_2011 = pd.read_csv('data/sales_2011.csv')
sales_2012 = pd.read_csv('data/sales_2012.csv')
sales_2013 = pd.read_csv('data/sales_2013.csv')
sales_2014 = pd.read_csv('data/sales_2014.csv')
sales_2015 = pd.read_csv('data/sales_2015.csv')
sales_2016 = pd.read_csv('data/sales_2016.csv')
sales_2017 = pd.read_csv('data/sales_2017.csv')
sales_2018 = pd.read_csv('data/sales_2018.csv')
sales_2019 = pd.read_csv('data/sales_2019.csv')
sales_2020 = pd.read_csv('data/sales_2020.csv')
sales_2021 = pd.read_csv('data/sales_2021.csv')

In [4]:
def ccleaner(data, date):
    data['Publish Date'] = date
    second_column = data.pop('Publish Date')
    data.insert(1, 'Publish Date', second_column)
    data['Price'] = data['Price'].replace('[\$,]', '', regex=True).astype(float)
    data['Est. units'] = data['Est. units'].replace(',', '', regex=True).astype(float)
    data['Total'] = data['Price'] * data['Est. units']
    data['Comic_ID'] = data.index
    
    data_list = []
    df = pd.DataFrame(columns=['Comic_ID', 'Publish Date', 'Comic-book Title', 'Price', 'Publisher', 'Est. units', 'Total'])
    
    for index, result in data.iterrows():
        local = {}
        local['Comic_ID'] = result['Comic_ID']
        local['Publish Date'] = result['Publish Date']
        local['Comic-book Title'] = result['Comic-book Title']
        local['Price'] = result['Price']
        local['Publisher'] = result['Publisher']
        local['Est. units'] = result['Est. units']
        local['Total'] = result['Total']
        data_list.append(local)
        
    df = pd.DataFrame(data_list)
    file_name = f'comic_{date}.csv'
    df.to_csv(f'data/{file_name}', index=False)   
    return df

In [5]:
def creator(data, check_list, date):
    matches = data[data['Comic-book Title'].str.contains('|'.join(check_list))]
    result_dict = {}
    for valor_b in check_list:
        matching_rows = matches[matches['Comic-book Title'].str.contains(valor_b)]
        if not matching_rows.empty:
            result_dict[valor_b] = {'Name': valor_b, 'Total': matching_rows['Total'].sum()} 
    result_df = pd.DataFrame.from_dict(result_dict, orient='index').reset_index()
    result_df.columns = ['Character_Name', 'Character_ID', 'Total']
    result_df['Character_ID'] = result_df.index
    result_df['Date'] = date
    result_df['Date'] =result_df['Date'].astype(int)
    result_df = result_df[['Character_ID', 'Date', 'Character_Name', 'Total']]
    file_name = f'characters_{date}.csv'
    result_df.to_csv(f'data/{file_name}', index=False)
    return result_df

In [6]:
def id_generator(df, check, date, id_name):
    identifier = []
    check_letter = {'c': 3, 'd': 4, 'h': 8, 'm': 13}
    identifier.append(check_letter.get(check, 0))
    last_two_digits = df[date] % 100
    identifier.append(last_two_digits.unique()[0])
    identifier.append(df.index)
    df[id_name] = identifier[0] * 10000 + identifier[1] * 100 + identifier[2]
    cols = list(df.columns)
    cols = [id_name] + cols[:-1]
    df = df[cols]
    return df

In [7]:
def save_csv(data, name):
    file_name = f'{name}.csv'
    data.to_csv(f'data/{file_name}', index=False)

In [8]:
def add_date(data):
    data['Release year'] = data['Release date(United States)'].str.extract(r'(\d{4})')
    data['Release year'] = data['Release year'].astype(int)
    data.drop('Release date(United States)', axis=1, inplace=True)
    second_column = data.pop('Release year')
    data.insert(2, 'Release year', second_column)
    return data

In [9]:
check_list = [
    'Absorbing Man', 'A.I.M.', 'Anole', 'Apocalypse', 'Archangel', 'Avalanche', 'Bastion', 'Beast',
    'Beyonder', 'Black Panther', 'Black Widow', 'Blob', 'Cable', 'Cannonball', 'Captain America',
    'Cassandra Nova', 'Chamber', 'Colossus', 'Crossbones', 'Cypher', 'Daredevil', 'Dark Phoenix',
    'Deadpool', 'Doctor Strange', 'Domino', 'Dust', 'Emma Frost', 'Exodus', 'Falcon', 'Fantomex',
    'Feral', 'Forge', 'Gambit', 'Gideon', 'Grandmaster', 'Grim Reaper', 'Havok', 'Hela', 'Hellion',
    'Hope Summers', 'Hulk', 'Iceman', 'Iron Man', 'Iron Patriot', 'Jean Grey', 'Jean Grey', 'Jubilee',
    'Juggernaut', 'Juggernaut', 'Kang', 'Karma', 'Korvac', 'Klaw', 'Lady Deathstrike', 'Living Laser',
    'Loki', 'Longshot', 'Luke Cage', 'M.O.D.O.K.', 'Magneto', 'Marrow', 'Mastermind', 'Master Mold',
    'Marauders', 'Marvel Boy', 'Multiple Man', 'Mystique', 'Nightcrawler', 'Omega Red', 'Onslaught',
    'Pixie', 'Polaris', 'Prodigy', 'Professor X', 'Psylocke', 'Red Skull', 'Rhino', 'Rockslide',
    'Rogue', 'Scarlet Witch', 'Sebastian Shaw', 'Selene', 'Shadowcat', 'Shadow King', 'Shatterstar',
    'Silver Samurai', 'Sinister Six', 'Skrulls', 'Sage', 'Spider-Man', 'Storm', 'Strong Guy', 'Sunspot',
    'Surge', 'Taskmaster', 'Thanos', 'The Hand', 'The Hood', 'Thor', 'U-Foes', 'Ultron', 'Vision',
    'Warpath', 'Wasp', 'William Stryker', 'Wolverine', 'Wolverine', 'X-23','X-Men', 'Avengers',
    'Ghost Rider', 'Venom', 'Dark Phoenix', 'Inhumans', 'Guardians of the Galaxy', 'Logan', 'Fantastic Four',
    'Punisher'
]




In [10]:
df_08 = ccleaner(sales_2008,'2008')
df_09 = ccleaner(sales_2009, '2009')
df_10 = ccleaner(sales_2010, '2010')
df_11 = ccleaner(sales_2011, '2011')
df_12 = ccleaner(sales_2012, '2012')
df_13 = ccleaner(sales_2013, '2013')
df_14 = ccleaner(sales_2014, '2014')
df_15 = ccleaner(sales_2015, '2015')
df_16 = ccleaner(sales_2016, '2016')
df_17 = ccleaner(sales_2017, '2017')
df_18 = ccleaner(sales_2018, '2018')
df_19 = ccleaner(sales_2019, '2019')
df_20 = ccleaner(sales_2020, '2020')
df_21 = ccleaner(sales_2021, '2021')

In [11]:
creator(df_08, check_list, '2008')
characters = creator(df_09, check_list, '2009')
creator(df_10, check_list, '2010')
creator(df_11, check_list, '2011')
creator(df_12, check_list, '2012')
creator(df_13, check_list, '2013')
creator(df_14, check_list, '2014')
creator(df_15, check_list, '2015')
creator(df_16, check_list, '2016')
creator(df_17, check_list, '2017')
creator(df_18, check_list, '2018')
creator(df_19, check_list, '2019')
creator(df_20, check_list, '2020')
creator(df_21, check_list, '2021')
characters

,Character_ID,Date,Character_Name,Total
0,0,2009,Black Panther,214662.0
1,1,2009,Cable,701665.0
2,2,2009,Captain America,4948275.0
3,3,2009,Daredevil,833181.0
4,4,2009,Deadpool,2702614.0
5,5,2009,Exodus,310821.0
6,6,2009,Hulk,5162349.0
7,7,2009,Iron Man,2158049.0
8,8,2009,Spider-Man,12278664.0
9,9,2009,Thor,2143686.0


In [12]:
characters.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Character_ID    16 non-null     int64  
 1   Date            16 non-null     int32  
 2   Character_Name  16 non-null     object 
 3   Total           16 non-null     float64
dtypes: float64(1), int32(1), int64(1), object(1)
memory usage: 580.0+ bytes


In [20]:
characters = id_generator(characters, 'c', 'Date', 'Character_ID')

In [14]:
movies = add_date(movies)

In [15]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Data columns (total 8 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Title                           46 non-null     object
 1   Distributor(s)                  46 non-null     object
 2   Release year                    46 non-null     int32 
 3   Budget (millions)               45 non-null     object
 4   Opening weekend(North America)  46 non-null     object
 5   North America                   46 non-null     object
 6   Other territories               46 non-null     object
 7   Worldwide                       46 non-null     object
dtypes: int32(1), object(7)
memory usage: 2.8+ KB


In [16]:
movies = id_generator(movies, 'm', 'Release year', 'Movie_ID')

In [17]:
save_csv(movies, 'movie box office')